# Import libraries

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F
import os

# Create spark session

In [2]:
if 'spark' in locals() and spark!=None:
    spark.stop()



spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 10:55:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Select chromosome

In [5]:
chromosome=None#"chr22"

if "which_chr" in os.environ:
    chromosome=os.environ['which_chr']
else:
    print("error : did not find which chromosome we are supposed to crunch")
    exit(-1)

# Load data

In [3]:
#define the phylop tsv schema
phylop_schema = StructType([
    StructField("CHROM", StringType(), True),
    StructField("POS", IntegerType(), True),
    StructField("P_ANNO", FloatType(), True),
])

#read in the phylop tsv
phylop_anno = spark.read \
    .option("comment", "#") \
    .option("delimiter", "\t") \
    .schema(phylop_schema) \
    .csv("/home/mcn26/varef/scripts/noon_data/1.0.format_zoonomia_phylop/out_processed.tsv", header=False)

In [6]:
vcf_schema = StructType([
    StructField("CHROM", StringType(), True),
    StructField("POS", IntegerType(), True),
    StructField("ID", StringType(), True),
    StructField("REF", StringType(), True),
    StructField("ALT", StringType(), True),
    StructField("QUAL", StringType(), True),
    StructField("FILTER", StringType(), True),
    StructField("INFO", StringType(), True),

])

vcf = spark.read \
    .option("comment", "#") \
    .option("delimiter", "\t") \
    .schema(vcf_schema) \
    .csv(f"/home/mcn26/varef/scripts/noon_data/0.merge/combined.{chromosome}.vcf.gz", header=False)


# Perform an in-memory merge

In [7]:
phyloP_annotated = vcf.join(phylop_anno, on=["CHROM", "POS"], how="inner")

# Output results

In [10]:
print(f"[+] Processing {chromosome}")
print(f"[+] Number of rows gnomad before merge: {vcf.count()}")
uniq_b4=vcf.select("CHROM", "POS").distinct().count()
print(f"[+] Number of unique chr, pos combos before merge {uniq_b4}")
print(f"[+] Number of rows gnomad after merge: {phyloP_annotated.count()}")
uniq_after=phyloP_annotated.select("CHROM", "POS").distinct().count()
print(f"[+] Number of unique chr, pos combos after merge {uniq_after}")

ERROR:root:KeyboardInterrupt while sending command.               (3 + 1) / 444]
Traceback (most recent call last):
  File "/gpfs/gibbs/project/reilly/mcn26/conda_envs/mcn_varef/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/gpfs/gibbs/project/reilly/mcn26/conda_envs/mcn_varef/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/gpfs/gibbs/project/reilly/mcn26/conda_envs/mcn_varef/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

# End spark session

In [ ]:
spark.stop()